In [15]:
import tkinter  
import tkinter.filedialog
root = tkinter.Tk()
z=""
def print_path():  
    global z
    f = tkinter.filedialog.askopenfilename(
        parent=root, initialdir=r'C:\Users\PRATYUSH\Desktop\ml\handwritten-MNIST-digit-recognition',
        title='Choose file',
        filetypes=[('jpeg images', '.jpg'),
                   ('gif images', '.gif')]
        )
    z=f
    
b1 = tkinter.Button(root, text='open and save path and display image', command=print_path)  
b1.pack(fill='x')
 
root.mainloop() 

In [16]:
# Import the modules
import cv2
from sklearn.externals import joblib
from skimage.feature import hog
import numpy as np

In [17]:
# Load the classifier
clf,pp = joblib.load("digits_cls1.pkl")

In [19]:
# Read the input image 
im = cv2.imread(z)
r = 3000.0 / im.shape[1]
dim = (4000, int(im.shape[0] * r))
str1=" "
# perform the actual resizing of the image and show it
im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)

In [20]:
# Convert to grayscale and apply Gaussian filtering
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)

In [21]:
# Threshold the image
ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)
# Find contours in the image
ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [22]:
# Get rectangles contains each contour
rects = [cv2.boundingRect(ctr) for ctr in ctrs]

In [23]:
for rect in rects:
    if rect[2] >30 and rect[3] >30:
        
        # Draw the rectangles
        cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
        # Make the rectangular region around the digit
        leng = int(rect[3] * 1.6)
        pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
        pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
        roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
        # Resize the image
        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
        roi = cv2.dilate(roi, (3, 3))
        # Calculate the HOG features
        roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False)
        roi_hog_fd = pp.transform(np.array([roi_hog_fd], 'float64'))
        # Pridict digits
        nbr = clf.predict(roi_hog_fd)
        
        cv2.putText(im, str(int(nbr[0])), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
        str1= str1+" "+ str(int(nbr[0]));
        print(str(int(nbr[0])))
            

6
7
5
4
3
1
2


In [24]:
cv2.namedWindow("Resulting Image with Rectangular ROIs", cv2.WINDOW_NORMAL)
cv2.imshow("Resulting Image with Rectangular ROIs", im)
cv2.waitKey()
cv2.imwrite('pred.jpg',im)

True

In [25]:
print(str1)
file1 = open("op.txt","w") 
file1.write(str1)
file1.close()

  6 7 5 4 3 1 2


In [26]:
from PIL import Image
import numpy as np

im1 = Image.open('1.jpg')
im2 = Image.open('pred.jpg')

Image.fromarray(np.hstack((np.array(im1)))).show()